In [92]:
import pandas as pd
import numpy as np
import cbbpy.mens_scraper as s
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

In [26]:
# game_info_df, boxscore_df, pbp_df = s.get_games_season(2024)

In [182]:
# try:
# game_info_df = pd.read_csv('raw_data/game_info_df.csv')
# boxscore_df = pd.read_csv('raw_data/boxscore_df.csv')
    # pbp_df = pd.read_csv('raw_data/pbp_df.csv')
# except:
game_info_df, boxscore_df, pbp_df = s.get_games_season(2024)
#     game_info_df.to_csv('game_info_df.csv',index=False)
#     boxscore_df.to_csv('boxscore_df.csv',index=False)
#     pbp_df.to_csv('pbp_df.csv',index=False)


Scraping 16 games on 03/21/24: 100%|██████████| 142 of 142 days scraped in 559.0 sec 


In [183]:
game_info_df.to_csv('raw_data/game_info_df.csv',index=False)
boxscore_df.to_csv('raw_data/boxscore_df.csv',index=False)
pbp_df.to_csv('raw_data/pbp_df.csv',index=False)

In [184]:
game_info_df.shape

(6148, 27)

In [105]:
# max_date = pd.to_datetime(game_info_df['game_day']).max()
# max_date = max_date.strftime('%Y-%m-%d')
# max_date
# new_date = '2024-03-01'
# new_game_info,new_boxscore_df,new_pbp_df = s.get_games_range(
#     start_date=new_date,
#     end_date = max_date,
#     info=True,
#     box=True,
#     pbp=False)

Scraping 22 games on 03/04/24: 100%|██████████| 4 of 4 days scraped in 15.3 sec 


## Summary

Assumptions:
* Refs, networks, arena capacity, date, time of day and tournament do not impact score

## Games

### Filter

In [109]:
game_info_df.shape

(5621, 27)

In [110]:
game_info_df.head()

,game_id,home_team,home_id,home_rank,home_record,home_score,away_team,away_id,away_rank,away_record,...,game_day,game_time,game_loc,arena,arena_capacity,attendance,tv_network,referee_1,referee_2,referee_3
0,401575451,Kansas Jayhawks,2305,1.0,1-0,99,North Carolina Central Eagles,2428,NaN,0-1,...,"November 06, 2023",05:00 PM PST,"Lawrence, KS",Allen Fieldhouse,0,16300,BIG12|ESPN+,Gerry Pollard,Amy Bonner,Chance Moore
1,401576147,Duke Blue Devils,150,2.0,1-0,92,Dartmouth Big Green,159,NaN,0-1,...,"November 06, 2023",06:00 PM PST,"Durham, NC",Cameron Indoor Stadium,0,9314,ACCN,Jamie Luckie,Ryan Sassano,Clare Aubry
2,401577640,Purdue Boilermakers,2509,3.0,1-0,98,Samford Bulldogs,2535,NaN,0-1,...,"November 06, 2023",03:30 PM PST,"West Lafayette, IN",Mackey Arena,0,14876,BTN,D.J. Carstensen,Edwin Young,Jourdan Love
3,401581835,Michigan State Spartans,127,4.0,0-1,76,James Madison Dukes,256,NaN,1-0,...,"November 06, 2023",05:30 PM PST,"East Lansing, MI",Breslin Center,0,14797,BTN,Brian Dorsey,Chad Barlow,Brian McNutt
4,401577598,Marquette Golden Eagles,269,5.0,1-0,92,Northern Illinois Huskies,2459,NaN,0-1,...,"November 06, 2023",05:30 PM PST,"Milwaukee, WI",Fiserv Forum,0,16352,,Lamar Simpson,Peter Larson,Ed Corliss


In [111]:
game_info_df.columns

Index(['game_id', 'home_team', 'home_id', 'home_rank', 'home_record',
       'home_score', 'away_team', 'away_id', 'away_rank', 'away_record',
       'away_score', 'home_win', 'num_ots', 'is_conference', 'is_neutral',
       'is_postseason', 'tournament', 'game_day', 'game_time', 'game_loc',
       'arena', 'arena_capacity', 'attendance', 'tv_network', 'referee_1',
       'referee_2', 'referee_3'],
      dtype='object')

In [188]:
print('Shape before filtering:', game_info_df.shape)
# filtered_df = game_info_df[game_info_df['is_postseason']==False]
filtered_df = game_info_df.copy()
filtered_df = filtered_df[(game_info_df['home_record']!='') & (game_info_df['away_record']!='')]
filtered_df = filtered_df.dropna(subset=['home_record','away_record'])
print('Shape after filtering:', filtered_df.shape)

Shape before filtering: (6148, 27)
Shape after filtering: (5638, 27)


In [189]:
exclude_cols = ['home_win','home_score', 'away_score', 'home_id', 'away_id','num_ots', 'is_postseason', 'tournament', 'game_time', 'game_loc',
       'arena', 'arena_capacity', 'attendance', 'tv_network', 'referee_1',
       'referee_2', 'referee_3']
       
date_col = 'game_day'
onehot_cols = ['home_team','away_team']
bool_cols = ['is_conference','is_neutral']
rank_cols = ['home_rank','away_rank']
records_cols = ['home_record','away_record']
# target_cols = ['home_score','away_score']


### Transform

In [190]:
# Exclude Cols
transformed_df = filtered_df.drop(exclude_cols, axis=1)

# Date col
transformed_df['game_day'] = pd.to_datetime(transformed_df['game_day'])

# One Hot Encode
# transformed_df = pd.get_dummies(transformed_df, columns=onehot_cols, prefix = ['home','away'], dtype=int)

# Convert Bool to Int
for col in bool_cols:
    transformed_df[col] = transformed_df[col].astype(int)

# Replace NULL Ranks with 99
for col in rank_cols:
    transformed_df[col] = transformed_df[col].fillna(99)

# Conver Record cols to Wins and Losses cols
for col in records_cols:
    team_type = col.split('_')[0]
    transformed_df[col] = transformed_df[col].astype(str)
    wins = transformed_df.loc[:,col].apply(lambda x: x.split('-')[0])
    losses = transformed_df.loc[:,col].apply(lambda x: x.split('-')[1])    
    transformed_df[team_type+'Team_wins'] = wins
    transformed_df[team_type+'Team_losses'] = losses
    transformed_df = transformed_df.drop(col,axis=1)

print(transformed_df.shape)

(5638, 12)


### Transformed Preview

In [191]:
transformed_df.head()

,game_id,home_team,home_rank,away_team,away_rank,is_conference,is_neutral,game_day,homeTeam_wins,homeTeam_losses,awayTeam_wins,awayTeam_losses
0,401575451,Kansas Jayhawks,1.0,North Carolina Central Eagles,99.0,0,0,2023-11-06,1,0,0,1
1,401576147,Duke Blue Devils,2.0,Dartmouth Big Green,99.0,0,0,2023-11-06,1,0,0,1
2,401577640,Purdue Boilermakers,3.0,Samford Bulldogs,99.0,0,0,2023-11-06,1,0,0,1
3,401581835,Michigan State Spartans,4.0,James Madison Dukes,99.0,0,0,2023-11-06,0,1,1,0
4,401577598,Marquette Golden Eagles,5.0,Northern Illinois Huskies,99.0,0,0,2023-11-06,1,0,0,1


## Boxscores

In [119]:
#TODO: convert home and away teams to vectors in transformed df?
#TODO; convert boxscores to vectors
    # TODO: find top 7-8 players by min played and find avg stats using those players and not whole team

In [120]:
boxscore_df.columns

Index(['game_id', 'team', 'player', 'player_id', 'position', 'starter', 'min',
       'fgm', 'fga', '2pm', '2pa', '3pm', '3pa', 'ftm', 'fta', 'oreb', 'dreb',
       'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts'],
      dtype='object')

In [192]:
stat_cols = ['fgm', 'fga', '2pm', '2pa', '3pm', '3pa', 'ftm', 'fta', 'oreb', 'dreb',
       'reb', 'ast', 'stl', 'blk', 'to', 'pf']
exclude_boxscore_cols = ['player', 'player_id', 'position', 'starter', 'min']


### Filter to Boxscore Totals

In [193]:
boxscores_filtered = boxscore_df[(boxscore_df.player_id == 'TOTAL')]

### Normalize Boxscore Stats

In [194]:
# Normalize Stats
scaler = MinMaxScaler()
boxscores_filtered[stat_cols] = scaler.fit_transform(boxscores_filtered[stat_cols])

/var/folders/z_/b2p80qvn353dbq0c1vfq0q600000gq/T/ipykernel_7923/2050784139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxscores_filtered[stat_cols] = scaler.fit_transform(boxscores_filtered[stat_cols])


In [195]:
boxscores_filtered.head()

,game_id,team,player,player_id,position,starter,min,fgm,fga,2pm,...,fta,oreb,dreb,reb,ast,stl,blk,to,pf,pts
12,401575451,North Carolina Central Eagles,TEAM,TOTAL,TOTAL,False,NaN,0.363636,0.558824,0.285714,...,0.352941,0.272727,0.250000,0.2875,0.189189,0.304348,0.055556,0.257143,0.325,56
25,401575451,Kansas Jayhawks,TEAM,TOTAL,TOTAL,False,NaN,0.709091,0.549020,0.530612,...,0.196078,0.121212,0.589286,0.4625,0.918919,0.173913,0.166667,0.285714,0.325,99
41,401576147,Dartmouth Big Green,TEAM,TOTAL,TOTAL,False,NaN,0.345455,0.549020,0.306122,...,0.333333,0.303030,0.339286,0.3625,0.297297,0.217391,0.055556,0.371429,0.400,54
54,401576147,Duke Blue Devils,TEAM,TOTAL,TOTAL,False,NaN,0.690909,0.578431,0.632653,...,0.235294,0.151515,0.500000,0.4125,0.486486,0.260870,0.333333,0.257143,0.450,92
68,401577640,Samford Bulldogs,TEAM,TOTAL,TOTAL,False,NaN,0.309091,0.666667,0.244898,...,0.274510,0.424242,0.339286,0.4125,0.297297,0.478261,0.000000,0.428571,0.475,45


### Convert Stat Columns into One stats_vector

In [196]:
# Get top 8 guys by minutes. Convert their stats into matrices. For input when predicting, will need to find averages and fill in each value
# OR just get their totals?
# boxscores_filtered['stats_vector'] = list(np.array(boxscores_filtered.loc[:,stat_cols]))
# bs_v = boxscores_filtered.drop(stat_cols+exclude_boxscore_cols,axis=1).reset_index(drop=True)
# bs_v
# test_boxscore.assign(new=pd.factorize(df.))

# Ignore vectors for now... 
bs_v = boxscores_filtered.drop(exclude_boxscore_cols,axis=1).reset_index(drop=True)
bs=bs_v.copy()

In [126]:
bs.shape

(11242, 19)

In [197]:
# df_rolling  = df_rolling.groupby(['team'], group_keys=False).apply(find_team_averages)
# Use cumulative instead of rolling
def find_team_averages(team):
    rolling = team.expanding().mean()
    return rolling

def shift_score(team):
    team['prev_scores_cum_avg'] = team['score'].expanding().mean()
    team['prev_scores_cum_avg'] = team['prev_scores_cum_avg'].shift(1)
    return team


In [198]:
# Join in transformed on game ID and the teams

include_cols = ['game_id','team','pts']+stat_cols
home_boxscores_df = transformed_df.rename(columns={'home_team':'team'})
home_boxscores_df['home_team']=home_boxscores_df['team']


away_boxscores_df = transformed_df.rename(columns={'away_team':'team'})
away_boxscores_df['away_team']=away_boxscores_df['team']

home_boxscores = pd.merge(bs, home_boxscores_df, how='left', on=['game_id','team'], validate='one_to_one').dropna()
# home_boxscores = home_boxscores.drop(['team'],axis=1)
away_boxscores = pd.merge(bs, away_boxscores_df, how='left', on=['game_id','team'], validate='one_to_one').dropna()[include_cols]
# combined_boxscores = pd.merge(home_boxscores,away_boxscores,on='game_id',validate='one_to_one',suffixes=('_home','_away'))
combined_boxscores = pd.merge(home_boxscores,away_boxscores,on='game_id',validate='one_to_one',suffixes=('_home','_away'))
combined_boxscores.head()

,game_id,team_home,fgm_home,fga_home,2pm_home,2pa_home,3pm_home,3pa_home,ftm_home,fta_home,...,ftm_away,fta_away,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,to_away,pf_away
0,401575451,Kansas Jayhawks,0.709091,0.549020,0.530612,0.440000,0.590909,0.442308,0.200,0.196078,...,0.250,0.352941,0.272727,0.250000,0.2875,0.189189,0.304348,0.055556,0.257143,0.325
1,401576147,Duke Blue Devils,0.690909,0.578431,0.632653,0.560000,0.318182,0.326923,0.225,0.235294,...,0.300,0.333333,0.303030,0.339286,0.3625,0.297297,0.217391,0.055556,0.371429,0.400
2,401577640,Purdue Boilermakers,0.618182,0.607843,0.367347,0.440000,0.727273,0.557692,0.350,0.431373,...,0.150,0.274510,0.424242,0.339286,0.4125,0.297297,0.478261,0.000000,0.428571,0.475
3,401581835,Michigan State Spartans,0.472727,0.705882,0.510204,0.693333,0.045455,0.384615,0.575,0.725490,...,0.525,0.549020,0.424242,0.660714,0.6375,0.378378,0.217391,0.222222,0.428571,0.800
4,401577598,Marquette Golden Eagles,0.636364,0.656863,0.530612,0.560000,0.409091,0.480769,0.325,0.411765,...,0.275,0.313725,0.545455,0.392857,0.5000,0.270270,0.260870,0.222222,0.400000,0.325


In [199]:
print(combined_boxscores.columns)
print(combined_boxscores.shape)

Index(['game_id', 'team_home', 'fgm_home', 'fga_home', '2pm_home', '2pa_home',
       '3pm_home', '3pa_home', 'ftm_home', 'fta_home', 'oreb_home',
       'dreb_home', 'reb_home', 'ast_home', 'stl_home', 'blk_home', 'to_home',
       'pf_home', 'pts_home', 'home_rank', 'away_team', 'away_rank',
       'is_conference', 'is_neutral', 'game_day', 'homeTeam_wins',
       'homeTeam_losses', 'awayTeam_wins', 'awayTeam_losses', 'home_team',
       'team_away', 'pts_away', 'fgm_away', 'fga_away', '2pm_away', '2pa_away',
       '3pm_away', '3pa_away', 'ftm_away', 'fta_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'to_away',
       'pf_away'],
      dtype='object')
(5638, 48)


In [200]:
def find_team_starters(team):
    team['num_starts'] = team['starter'].expanding().sum()
    team['num_starts'] = team['num_starts'].shift(1).fillna(1)
    return team

boxscore_df['starter'] = boxscore_df['starter'].astype(int)

rolling_num_starts_df = boxscore_df.groupby(['team','player'],group_keys=False)[['starter']].apply(find_team_starters)#.rename(columns={'starter':''})
boxscores_with_rolling_num_starts = pd.concat([boxscore_df, rolling_num_starts_df],axis=1)
# boxscores_with_rolling_num_starts.head()
# final_df = pd.merge(combined_boxscores, boxscores_with_rolling_num_starts, on=['game_id','team_home','player'])

In [201]:
#TODO need to find if prev game had all their starters. 
# Join in one column to final dataset. Either 
# the num starters played or arr of starters

# Need to find array of starters and shift them forward
# GROUP BY GAME ID FIND TOP 5 NUM_STARTS
boxscores_with_rolling_num_starts.groupby(['game_id','team'],group_keys=False) \
                                 .aggregate({'player':'count'})
                                

player
game_id   team                               
401573353 Eastern Washington Eagles        10
          Montana Grizzlies                10
401573354 Idaho Vandals                    11
          Montana State Bobcats            10
401573355 Idaho State Bengals              10
...                                       ...
401648806 Tarleton State Texans            12
401648807 Bowling Green Falcons             9
          Purdue Fort Wayne Mastodons      10
401648808 Alabama A&M Bulldogs             12
          Austin Peay Governors             9

[12296 rows x 1 columns]

In [202]:
combined_boxscores.to_csv('processed_data/combined_boxscores.csv',index=False)

### Add Starters